In [1]:
import geopandas as gpd

/home/jupyter-peta_admin/.local/lib/python3.8/site-packages/geopandas/_compat.py:112: UserWarning: The Shapely GEOS version (3.10.2-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


In [2]:
kdprov='19'

In [3]:
data_gpd=gpd.read_file("data/output/1_population_density/grid_pop_"+kdprov+"_1km_w_bu.gpkg"
                      ).reset_index()

In [4]:
data_gpd['Name']=data_gpd.index.astype(str).str.zfill(7)

In [5]:
##Check cluster centre
data_cl_centre=data_gpd.query('pop>=1500')

In [6]:
from pysal.lib import weights

In [7]:
wr = weights.contiguity.Rook.from_dataframe(data_cl_centre)
data_cl_centre['cluster_label']=wr.component_labels

/home/jupyter-peta_admin/.local/lib/python3.8/site-packages/libpysal/weights/weights.py:172: UserWarning: The weights matrix is not fully connected: 
 There are 71 disconnected components.
 There are 50 islands with ids: 7, 8, 9, 10, 13, 16, 17, 18, 19, 22, 23, 24, 25, 26, 27, 28, 29, 31, 34, 36, 37, 38, 39, 40, 43, 44, 45, 46, 48, 51, 56, 66, 109, 113, 117, 118, 121, 122, 123, 124, 127, 128, 132, 133, 138, 147, 149, 150, 151, 174.
  warnings.warn(message)
/home/jupyter-peta_admin/.local/lib/python3.8/site-packages/geopandas/geodataframe.py:1456: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [8]:
import pandas as pd

In [9]:
tab_pop_u_centre_cand=pd.pivot_table(data_cl_centre,
                                     index='cluster_label',values='pop',aggfunc='sum')

In [10]:
list_u_centre=tab_pop_u_centre_cand.query('pop>50000').reset_index().cluster_label.tolist()

In [11]:
u_centre=data_cl_centre.query('cluster_label in @list_u_centre')

In [12]:
data_gpd['is_U_CENTRE']=data_gpd.Name.apply(lambda y: 1 if y in u_centre.Name.tolist() else 0)

In [13]:
#Filling gap of urbancentre => check 8 contiguity (queen)
init_u_centre=data_gpd.is_U_CENTRE.sum()
u_centre_up=0
print('N GRID WITH URBAN CENTRE:',init_u_centre)
while u_centre_up!=init_u_centre:
    init_u_centre=data_gpd.is_U_CENTRE.sum()
    wr = weights.contiguity.Queen.from_dataframe(data_gpd)
    data_gpd['neighbors']=wr.neighbors.values()
    temp=data_gpd.copy()
    for index, row in data_gpd.iterrows():
        if row['is_U_CENTRE']!=1:
            y_urbancentre=0
            if len(row['neighbors'])>4:
                for y in row['neighbors']:
                    if temp.loc[y,'is_U_CENTRE']==1:
                        y_urbancentre+=1
                if y_urbancentre>4:
                    data_gpd.at[index,'is_U_CENTRE']=1
    u_centre_up=data_gpd.is_U_CENTRE.sum()
    print('=====================')
    print('N GRID AFTER GAP FILLING:', u_centre_up)
print('==================')
u_centre=data_gpd.is_U_CENTRE.sum()
print('N GRID COMPLETED AFTER GAP FILLING',u_centre)

N GRID WITH URBAN CENTRE: 66


/home/jupyter-peta_admin/.local/lib/python3.8/site-packages/libpysal/weights/weights.py:172: UserWarning: The weights matrix is not fully connected: 
 There are 80 disconnected components.
 There are 24 islands with ids: 514, 7668, 10501, 12586, 12660, 12685, 12686, 12698, 12893, 12894, 13093, 14029, 14033, 15413, 16727, 17891, 18089, 18128, 18146, 18154, 18173, 18197, 18201, 18226.
  warnings.warn(message)


N GRID AFTER GAP FILLING: 70


/home/jupyter-peta_admin/.local/lib/python3.8/site-packages/libpysal/weights/weights.py:172: UserWarning: The weights matrix is not fully connected: 
 There are 80 disconnected components.
 There are 24 islands with ids: 514, 7668, 10501, 12586, 12660, 12685, 12686, 12698, 12893, 12894, 13093, 14029, 14033, 15413, 16727, 17891, 18089, 18128, 18146, 18154, 18173, 18197, 18201, 18226.
  warnings.warn(message)


N GRID AFTER GAP FILLING: 71


/home/jupyter-peta_admin/.local/lib/python3.8/site-packages/libpysal/weights/weights.py:172: UserWarning: The weights matrix is not fully connected: 
 There are 80 disconnected components.
 There are 24 islands with ids: 514, 7668, 10501, 12586, 12660, 12685, 12686, 12698, 12893, 12894, 13093, 14029, 14033, 15413, 16727, 17891, 18089, 18128, 18146, 18154, 18173, 18197, 18201, 18226.
  warnings.warn(message)


N GRID AFTER GAP FILLING: 72


/home/jupyter-peta_admin/.local/lib/python3.8/site-packages/libpysal/weights/weights.py:172: UserWarning: The weights matrix is not fully connected: 
 There are 80 disconnected components.
 There are 24 islands with ids: 514, 7668, 10501, 12586, 12660, 12685, 12686, 12698, 12893, 12894, 13093, 14029, 14033, 15413, 16727, 17891, 18089, 18128, 18146, 18154, 18173, 18197, 18201, 18226.
  warnings.warn(message)


N GRID AFTER GAP FILLING: 72
N GRID COMPLETED AFTER GAP FILLING 72


In [14]:
data_gpd['is_U_CENTRE_w_BU']=data_gpd.apply(lambda y:1 if (y.p_builtup>0.5)or(y.is_U_CENTRE==1) else 0,axis=1)

In [15]:
data_gpd=data_gpd[['Name','pop','geometry','p_builtup','is_U_CENTRE_w_BU']]

In [16]:
data_gpd

,Name,pop,geometry,p_builtup,is_U_CENTRE_w_BU
0,0000000,0,"POLYGON ((10531200.000 -248400.000, 10531200.0...",0.000000,0
1,0000001,0,"POLYGON ((10531200.000 -249400.000, 10531200.0...",0.000000,0
2,0000002,0,"POLYGON ((10531200.000 -250400.000, 10531200.0...",0.000000,0
3,0000003,0,"POLYGON ((10531200.000 -251400.000, 10531200.0...",0.000000,0
4,0000004,0,"POLYGON ((10531200.000 -252400.000, 10531200.0...",0.000000,0
...,...,...,...,...,...
18242,0018242,0,"POLYGON ((10895200.000 -319400.000, 10895200.0...",0.000003,0
18243,0018243,0,"POLYGON ((10901200.000 -311400.000, 10901200.0...",0.000000,0
18244,0018244,0,"POLYGON ((10901200.000 -312400.000, 10901200.0...",0.000000,0
18245,0018245,0,"POLYGON ((10903200.000 -308400.000, 10903200.0...",0.000000,0


In [17]:
data_u_cluster=data_gpd.query('is_U_CENTRE_w_BU!=1').query('pop>=300')

In [18]:
wr = weights.contiguity.Queen.from_dataframe(data_u_cluster)
data_u_cluster['cluster_label']=wr.component_labels

/home/jupyter-peta_admin/.local/lib/python3.8/site-packages/libpysal/weights/weights.py:172: UserWarning: The weights matrix is not fully connected: 
 There are 285 disconnected components.
 There are 115 islands with ids: 0, 26, 30, 38, 54, 57, 59, 62, 66, 72, 84, 104, 105, 114, 115, 118, 131, 132, 153, 154, 162, 175, 180, 188, 191, 201, 202, 212, 213, 228, 231, 237, 240, 248, 251, 252, 253, 254, 257, 261, 271, 279, 289, 297, 301, 304, 308, 320, 326, 330, 335, 351, 353, 359, 360, 368, 371, 373, 479, 480, 510, 591, 601, 611, 612, 639, 651, 658, 660, 661, 671, 672, 716, 750, 766, 770, 774, 778, 781, 782, 783, 797, 804, 807, 815, 816, 817, 825, 826, 830, 834, 843, 881, 884, 885, 890, 891, 892, 894, 896, 899, 901, 905, 911, 926, 928, 929, 931, 938, 939, 941, 944, 974, 1001, 1019.
  warnings.warn(message)


In [19]:
tab_pop_u_cluster_cand=pd.pivot_table(data_u_cluster,
                                     index='cluster_label',values='pop',aggfunc='sum')
list_u_cluster=tab_pop_u_cluster_cand.query('pop>=5000').reset_index().cluster_label.tolist()

In [20]:
u_cluster=data_u_cluster.query('cluster_label in @list_u_cluster')

In [21]:
data_gpd['is_U_CLUSTER']=data_gpd.Name.apply(lambda y: 1 if y in u_cluster.Name.tolist() else 0)

In [22]:
data_gpd['classification']=data_gpd.apply(lambda y: 'CENTRE' 
                                          if y.is_U_CENTRE_w_BU==1 else ('CLUSTER' if y.is_U_CLUSTER==1 else 'RURAL'),axis=1)

In [23]:
data_gpd.classification.unique()

array(['RURAL', 'CLUSTER', 'CENTRE'], dtype=object)

In [25]:
data_gpd.to_file('data/output/2_grid_degurba/degurba_'+kdprov+'00.gpkg',driver='GPKG')